## Part 3 - Deploy the model

In the second notebook we created a basic model and exported it to a file. In this notebook we'll use that same model file to create a REST API with Microsoft ML Server. The Ubuntu DSVM has an installation of ML Server for testing deployments. We'll create a REST API with our model and test it with the same truck image we used in notebook 2 to evaluate the model. 

There are two variables you must set before running this notebook. The first is the password for your ML Server instance. At MLADS we've already set this for you. If you're following this tutorial on your own, you should configure your ML Server instance for [one-box deployment](https://docs.microsoft.com/en-us/machine-learning-server/operationalize/configure-machine-learning-server-one-box). The second variable is the name of the deployed web service. This needs to be unique on the VM. We recommend that you use your username and a number, like *username5*.

In [15]:
# choose a unique service name. We recommend you use your username and a number, like alias3
service_name = ____SET_ME_TO_A_UNIQUE_VALUE_____

# set the ML Server admin password
ml_server_password =  ____SET_ME_____ 

In [10]:
from azureml.deploy import DeployClient
from azureml.deploy.server import MLServer

HOST = 'http://localhost:12800'
context = ('admin', ml_server_password)
client = DeployClient(HOST, use=MLServer, auth=context)

Retrieve the truck image for testing our deployed service.

In [11]:
from PIL import Image
import pandas as pd
import numpy as np
from matplotlib.pyplot import imshow
from IPython.display import Image as ImageShow

try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib import urlopen

url = "https://cntk.ai/jup/201/00014.png"
myimg = np.array(Image.open(urlopen(url)), dtype=np.float32)
flattened = myimg.ravel()

ImageShow(url=url, width=64, height=64)

In [12]:
import cntk

with open('model.cntk', mode='rb') as file: # b is important -> binary
    binary_model = file.read()

# --Define an `init` function to handle service initialization --
def init():
    import cntk
    
# define an eval function to handle scoring
def eval(image_data):
    import numpy as np
    import cntk
    from pandas import DataFrame
    
    image_data = image_data.copy().reshape((32, 32, 3))
    
    image_mean = 133.0
    image_data -= image_mean
    image_data = np.ascontiguousarray(np.transpose(image_data, (2, 0, 1)))
    
    loaded_model = cntk.ops.functions.load_model(binary_model)    
    results = loaded_model.eval({loaded_model.arguments[0]:[image_data]})
        
    return DataFrame(results)

In [16]:
# create the API
service = client.service(service_name)\
        .version('1.0')\
        .code_fn(eval, init)\
        .inputs(image_data=np.array)\
        .outputs(results=pd.DataFrame)\
        .models(binary_model=binary_model)\
        .description('My CNTK model')\
        .deploy()
        
print(help(service))
service.capabilities()

Help on Txservice12Service in module azureml.deploy.server.service object:

class Txservice12Service(Service)
 |  Service object from metadata.
 |  
 |  Method resolution order:
 |      Txservice12Service
 |      Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, service, http_client)
 |      Constructor
 |      
 |      :param service:
 |      :param http_client:
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  batch(self, records, parallel_count=10)
 |      Register a set of input records for batch execution on this service.
 |      
 |      :param records: The `data.frame` or `list` of
 |             input records to execute.
 |      :param parallel_count: Number of threads used to process entries in
 |             the batch. Default value is 10. Please make sure not to use too
 |             high of a number because it might negatively impact performance.
 |      :return: The `Batch` object to control service batching
 |              lif

{'api': '/api/TxService12/1.0',
 'artifacts': [],
 'creation_time': '2018-05-23T04:14:18.851557',
 'description': 'My CNTK model',
 'inputs': [{'name': 'image_data', 'type': 'vector'}],
 'inputs_encoded': [{'name': 'image_data', 'type': 'numpy.array'}],
 'name': 'TxService12',
 'operation_id': 'eval',
 'outputs': [{'name': 'results', 'type': 'data.frame'}],
 'outputs_encoded': [{'name': 'results', 'type': 'pandas.DataFrame'}],
 'public-functions': {'batch': 'batch(records, parallel_count=10)',
  'capabilities': 'capabilities()',
  'eval': 'eval(self,image_data)',
  'get_batch': 'get_batch(execution_id)',
  'list_batch_execution': 'list_batch_execution()',
  'swagger': 'swagger(json=True)'},
 'published_by': 'admin',
 'runtime': 'Python',
 'snapshot_id': '370253b2-1e4a-4007-9890-3eb2b904508e',
 'swagger': 'http://localhost:12800/api/TxService12/1.0/swagger.json',
 'version': '1.0'}

Now call our newly created API with our truck image.

In [17]:
res = service.eval(flattened)

# -- Pluck out the named output `results` as defined during publishing and print --
print(res.output('results'))

# get the top 3 predictions
result = res.output('results')
result = result.as_matrix()[0]
top_count = 3
result_indices = (-np.array(result)).argsort()[:top_count]

label_lookup = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
print("Top 3 predictions:")
for i in range(top_count):
    print("\tLabel: {:10s}, confidence: {:.2f}%".format(label_lookup[result_indices[i]], result[result_indices[i]] * 100))

# -- Retrieve the URL of the swagger file for this service.
cap = service.capabilities()
swagger_URL = cap['swagger']
print(swagger_URL)

         0         1         2         3         4         5         6  \
0  0.10102  0.115993  0.098485  0.101911  0.093252  0.096514  0.097339   

          7        8         9  
0  0.099847  0.09435  0.101288  
Top 3 predictions:
	Label: automobile, confidence: 11.60%
	Label: cat       , confidence: 10.19%
	Label: truck     , confidence: 10.13%
http://localhost:12800/api/TxService12/1.0/swagger.json


In [15]:
print(service.swagger())
